In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document

# PDF 파일 경로 설정
pdf_file_path = '../data/sample.pdf'

# PDF에서 텍스트 추출
def load_pdf_document(pdf_file_path):
    loader = PyPDFLoader(pdf_file_path)
    documents = loader.load()
    return documents

# PDF 파일 로딩 및 문서 출력
pdf_documents = load_pdf_document(pdf_file_path)

# 첫 번째 문서 내용 일부 출력
print(pdf_documents[0].page_content[:1000])


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 82 0 (offset 0)
Ignoring wrong pointing object 162 0 (offset 0)
Ignoring wrong pointing object 211 0 (offset 0)
Ignoring wrong pointing object 272 0 (offset 0)
Ignoring wrong pointing object 284 0 (offset 0)
Ignoring wrong pointing object 302 0 (offset 0)
Ignoring wrong pointing object 314 0 (offset 0)
Ignoring wrong pointing object 381 0 (offset 0)
Ignoring wrong pointing object 407 0 (offset 0)
Ignoring wrong pointing object 415 0 (offset 0)


제 안 요 청 서
년 월2024 7
사업 책임자뉴스빅데이터팀팀장김선형TEL:02-2001-7571FAX:02-2001-7560사업 실무자뉴스빅데이터팀과장박상민TEL:02-2001-7573


In [28]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 웹 페이지에서 특정 HTML 요소를 로드하는 함수
def load_web_documents(urls):
    all_docs = []
    for url in urls:
        loader = WebBaseLoader(
            web_paths=(url,),
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    "div",
                    attrs={"class": ["NewsEndMain_article_head_title__ztaL4", "_article_content"]},
                )
            ),
        )
        docs = loader.load()
        all_docs.extend(docs)
        print(f"URL: {url} - 문서의 수: {len(docs)}")
        if docs:
            print(f"첫 문서 내용: {docs[0].page_content[:1000]}...")  # 첫 문서 내용 일부 출력
    return all_docs

# URL 리스트
urls = [
    "https://www.content.upstage.ai/blog/people/enterprise-business-solution-interview"
    # 추가 URL을 여기에 추가
]

# 웹 페이지에서 문서 로딩 및 출력
web_documents = load_web_documents(urls)


URL: https://www.content.upstage.ai/blog/people/enterprise-business-solution-interview - 문서의 수: 1
첫 문서 내용: ...


In [9]:
import os
from langsmith import Client, traceable
from dotenv import load_dotenv

# .env 파일에서 API 키를 불러옵니다.
load_dotenv()
api_key = os.getenv('LANGCHAIN_API_KEY')

# LangSmith 클라이언트 초기화
client = Client(api_key=api_key)

@traceable
def sample_function():
    # 트래킹을 위한 함수
    print("This function is being tracked by LangSmith")

sample_function()


This function is being tracked by LangSmith


In [12]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langsmith import Client, traceable

# 환경변수 로드
load_dotenv()

# LangSmith API 클라이언트 설정 (키는 .env 파일에서 가져옵니다)
client = Client(api_key=os.getenv('LANGCHAIN_API_KEY'))

print(api_key)

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


In [52]:
# PDF 로더
@traceable
def load_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    print(f"PDF 문서 로드 완료: {file_path} - 총 페이지 수: {len(documents)}")
    return documents

# 웹 페이지 로더
@traceable  # Auto-trace this function
def load_documents(urls):
    all_docs = []
    for url in urls:
        loader = WebBaseLoader(
            web_paths=(url,),
            bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(
                    "div",
                    attrs={"class": ["newsct_article _article_body", "media_end_head_title"]},  # 원하는 클래스명
                )
            ),
        )
        docs = loader.load()
        all_docs.extend(docs)
        print(f"URL: {url} - 문서의 수: {len(docs)}")
        
        if docs:
            # 첫 문서 내용 일부 출력
            print(f"첫 문서 내용: {docs[0].page_content[:1000]}...")
            
            # 클래스명에 해당하는 요소의 개수를 출력
            soup = bs4.BeautifulSoup(docs[0].page_content, 'html.parser')
            elements = soup.find_all("div", class_=["newsct_article _article_body", "media_end_head_title"])
            print(f"추출된 요소의 개수: {len(elements)}")
            if elements:
                print(f"첫 번째 요소의 내용: {elements[0].text[:200]}...")
    
    return all_docs

# 텍스트 청킹
@traceable
def chunk_documents(documents, chunk_size=1000, chunk_overlap=50):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(documents)
    print(f"청크된 문서 수: {len(chunks)}")
    return chunks

In [53]:
# 파일과 URL을 로드하고 청킹하는 예시
pdf_file_path = '../data/sample.pdf'  # PDF 파일 경로
urls = ["https://m.sports.naver.com/wbaseball/article/311/0001762982"]  # 웹 페이지 URL

# PDF 로딩 및 청킹
pdf_documents = load_pdf(pdf_file_path)
pdf_chunks = chunk_documents(pdf_documents)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 61 0 (offset 0)
Ignoring wrong pointing object 82 0 (offset 0)
Ignoring wrong pointing object 162 0 (offset 0)
Ignoring wrong pointing object 211 0 (offset 0)
Ignoring wrong pointing object 272 0 (offset 0)
Ignoring wrong pointing object 284 0 (offset 0)
Ignoring wrong pointing object 302 0 (offset 0)
Ignoring wrong pointing object 314 0 (offset 0)
Ignoring wrong pointing object 381 0 (offset 0)
Ignoring wrong pointing object 407 0 (offset 0)
Ignoring wrong pointing object 415 0 (offset 0)


PDF 문서 로드 완료: ../data/sample.pdf - 총 페이지 수: 141
청크된 문서 수: 160


In [54]:
urls = ["https://n.news.naver.com/article/008/0005080020"]  # 웹 페이지 URL
# 웹 페이지 로딩 및 청킹
web_documents = load_documents(urls)
web_chunks = chunk_documents(web_documents)

URL: https://n.news.naver.com/article/008/0005080020 - 문서의 수: 1
첫 문서 내용: 
"제발 SNS에 올리지 마"…백숙 먹는 손님 북적 '계곡 맛집'의 실체






/사진=보배드림충청도 한 계곡에서 불법 영업 중인 식당이 철거 행정 명령에도 꿋꿋하게 장사를 하는 현장이 공개됐다.지난 18일 한 온라인 커뮤니티에 'SNS에 제발 글 올리지 말라는 식당 근황'이라는 글이 게시됐다.이 식당은 계곡에 테이블과 의자를 두고 백숙, 삼겹살을 팔고 있다. 식당은 'SNS에 제발 사진 좀 올리지 말아주세요~~!!'라는 글을 쓴 커다란 현수막을 달고 장사를 했다.작성자 A씨는 "충북 진천 어느 식당에 가면 이런 문구를 붙여둔 곳이 있다"며 "요즘 같은 시대에 홍보하지 말아 달라는 건 홍보가 필요 없을 정도로 문전성시라 그런가 보다"라고 했다.그러면서 "얼마나 맛있는지 해당 지역 군청에 문의해 봤다"며 7월25일 민원을 넣은 사실을 알렸다. 민원은 불법영업 취지로 국민신문고에 낸 것으로 보인다.진상은 이달 1일 진천군청이 A씨에 업소 점검 결과를 알리면서 밝혀졌다. 군청은 "'○○(가게명)'은 영업 신고를 한 면적 외에 장소(계곡 내 테이블, 의자 설치)를 객석 등으로 사용해 영업하고 있었으며, 영업장 면적을 변경하고 변경 신고를 하지 않았으므로 「식품위생법」 제37조 위반으로 행정처분 진행 중임을 안내드린다"고 답변했다.이어 "행위자(식당 측)에게 7월29일까지 원상복구 명령 조치를 했으며, 미이행 시 관련 규정에 따라 고발 조치할 예정이다"며 "우리 군에서도 예찰 활동을 강화해 하천 내 불법 행위 단속을 철저하게 하겠다"고 알렸다.A씨에 따르면 이 식당은 원상복구 명령에 아랑곳하지 않고 영업을 이어갔다. 원상복구 날짜 7월29일을 넘긴 날짜에도 식당 후기가 잇따랐다. 심지어 8월 중 한 후기엔 "사람이 더 많아졌다"는 평도 있었다.누리꾼들은 "불법 영업으로 벌금을 내도 장사해서 버는 수익이 더 크니 계속 반복하는 거다", "

In [51]:
print(web_chunks)

[]
